In [76]:
from typing import List, Tuple
import pandas as pd
from argparser_config import get_parser

from get_returning_player_team_info import get_returning_team_stats
from create_weighted_player_stats import get_team_info

import helper_functions as hf

FUNCTION_TO_TABLE_DICT = {
    get_returning_team_stats: "returning_player_team_stats",
    get_team_info: "team_weighted_info_tourney",
}


def regenerate_failed_data(function, failed_team_season_lst):
    outputs = []
    for failed_team_season in failed_team_season_lst:
        team, season = failed_team_season
        output = function(team, season)
        outputs.append([team, *output, season])
    return pd.DataFrame(outputs)


def load_existing_data(function, year):
    file_path = f"{hf.get_generated_dir(year)}/{FUNCTION_TO_TABLE_DICT[function]}.csv"
    df = pd.read_csv(file_path)
    return df


def append_failed_data(function, failed_team_seasons_lst, data_year):
    failed_data = regenerate_failed_data(function, failed_team_seasons_lst)
    existing_data = load_existing_data(function, data_year)
    failed_data.columns = existing_data.columns
    return pd.concat([existing_data, failed_data], ignore_index=True).sort_values(
        by=["year", "team"]
    )

- Rerun `get_returning_team_stats` for updates to the returning player info table.
- Rerun `get_team_info` for updates to the weighted team info table.

## Failed Data: Weighted Info

In [99]:
year = 2022

In [100]:
existing_data = load_existing_data(get_team_info, year)
print(f"Existing data has {len(existing_data)} rows.")

Existing data has 1269 rows.


In [101]:
# The below teams failed while running the script due to API calls failing.
# They both succeed when run as one-offs.
failed_team_seasons_lst = [
    ['syracuse', 2012],
    ['minnesota', 2017]
]

In [102]:
df = append_failed_data(get_team_info, failed_team_seasons_lst, year)
print(f"New data has {len(df)} rows.")

New data has 1271 rows.


In [103]:
# write the dataframe to a csv
file_path = f"{hf.get_generated_dir(year)}/{FUNCTION_TO_TABLE_DICT[get_team_info]}.csv"
hf.write_to_csv(df, file_path, overwrite=False)

This file already exists. The overwrite flag is set to False so the existing file was not overwritten.
